# DVC Getting Started

В этом ноутбуке показаны основы работы с DVC на примере двух человек: Алисы 👩 и Боба 👨. 
Алиса настраивает DVC и создает пайплайн.
Боб воспроизводит результаты Алисы.

# Подготовка

Чтобы освоить dvc нам понадобится несколько репозиториев:
- один git-репозиторий, который будет выполнять роль remote/origin.
- 2 локальных git-репозитория - один для Алисы, другой для Боба.
- 1 dvc-хранилище.

Настроим перечисленные репозитории:

In [1]:
import os
from os.path import expanduser

# Библиотека для репозиториев 
ROOT_DIR = os.path.join(expanduser("~"), 'tmp_dvc')
GIT_REMOTE = os.path.join(ROOT_DIR, 'git_remote')
DVC_REMOTE = os.path.join(ROOT_DIR, 'dvc_remote')
ALICE_DIR = os.path.join(ROOT_DIR, 'Alice')
BOB_DIR = os.path.join(ROOT_DIR, 'Bob')

os.makedirs(GIT_REMOTE)
os.makedirs(DVC_REMOTE)
os.makedirs(ALICE_DIR)
os.makedirs(BOB_DIR)

In [2]:
# Initialize git remote repository
!git init --bare $GIT_REMOTE

Инициализирован пустой репозиторий Git в /home/nik/tmp_dvc/git_remote/


# Алиса

Инициализируем репозиторий для первого пользователя

In [3]:
!git clone $GIT_REMOTE $ALICE_DIR

Клонирование в «/home/nik/tmp_dvc/Alice»…
готово.


## Создание файлов для обучения

У первого пользователь создадим датасет для обучения.
Это будет CSV с датасетом вин (https://archive.ics.uci.edu/ml/datasets/wine+quality)
Позже мы добавим этот файл под контроль DVC.

In [4]:
csv_path = os.path.join(ALICE_DIR, 'winequality-red.csv')
!cp 'winequality-red.csv' $csv_path

Создадим в репозитории первого пользователя скрипт, который будет обучать модель:

In [5]:
script_path = os.path.join(ALICE_DIR, 'train_model.py')
!cp 'train_model.py' $script_path

## Настройка DVC

In [6]:
# Перейти в каталог первого пользователя
%cd $ALICE_DIR

# Инициализация dvc в репозитории
!dvc init

/home/nik/tmp_dvc/Alice
Adding '.dvc/state' to '.dvc/.gitignore'.
Adding '.dvc/lock' to '.dvc/.gitignore'.
Adding '.dvc/config.local' to '.dvc/.gitignore'.
Adding '.dvc/updater' to '.dvc/.gitignore'.
Adding '.dvc/updater.lock' to '.dvc/.gitignore'.
Adding '.dvc/repos' to '.dvc/.gitignore'.
Adding '.dvc/state-journal' to '.dvc/.gitignore'.
Adding '.dvc/state-wal' to '.dvc/.gitignore'.
Adding '.dvc/cache' to '.dvc/.gitignore'.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|              https://dvc.org/doc/user-guide/analytics               |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------

Если выполнить команду `git status`, то мы увидим, что появился каталог `.dvc` с файлами `config` и `.gitignore`

In [7]:
# Показать что изменилось в репозитории
!git status

На ветке master

Еще нет коммитов

Изменения, которые будут включены в коммит:
  (используйте «git rm --cached <файл>…», чтобы убрать из индекса)

	новый файл:    .dvc/.gitignore
	новый файл:    .dvc/config

Неотслеживаемые файлы:
  (используйте «git add <файл>…», чтобы добавить в то, что будет включено в коммит)

	train_model.py
	winequality-red.csv



Для работы dvc необходим удаленный репозиторий для хранения всех данных. Добавим его:

In [8]:
!dvc remote add myremote $DVC_REMOTE

Добавим наш датасет под контроль DVC:

In [9]:
!dvc add 'winequality-red.csv'

Adding 'winequality-red.csv' to '.gitignore'.
Saving 'winequality-red.csv' to '.dvc/cache/2d/aeecee174368f8a33b82c8cccae3a5'.
Saving information to 'winequality-red.csv.dvc'.

To track the changes with git, run:

	git add .gitignore winequality-red.csv.dvc


После `git add` появится новый файл 'winequality-red.csv.dvc', а csv-файл будет добавлен в `.gitignore`. При работе с DVCS именно файлы с расширением `*.dvc` должны находиться под контролем git, а реальные файлы должны храниться в кэше DVC и хранилище DVC.

Сохраним наш прогресс:

In [10]:
!git add .
!git commit -m "Initialized dataset"

[master (корневой коммит) a803109] Initialized dataset
 5 files changed, 73 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .gitignore
 create mode 100644 train_model.py
 create mode 100644 winequality-red.csv.dvc


Убедимся, что в remote-репозитории dvc наши данные отсутствуют:

In [11]:
!dvc status -r myremote

Preparing to collect status from ../dvc_remote
[##############################] 100% 

[##############################] 100% 

new:                winequality-red.csv


Запушим наш прогресс:

In [12]:
# "dvc push" обязателен! Иначе у пользователей не будет возможности получить файлы под контролем DVC!!!
!dvc push -r myremote

Preparing to upload data to '../dvc_remote'
Preparing to collect status from ../dvc_remote
[##############################] 100% 

[##############################] 100% 
[##############################] 100% Analysing status
[##############################] 100% winequality-red.csv
[##############################] 100% winequality-red.csv


In [13]:
!git push

Подсчет объектов: 8, готово.
Delta compression using up to 4 threads.
Сжатие объектов: 100% (7/7), готово.
Запись объектов: 100% (8/8), 1.27 KiB | 1.27 MiB/s, готово.
Total 8 (delta 0), reused 0 (delta 0)
To /home/nik/tmp_dvc/git_remote
 * [new branch]      master -> master


## Обновление файлов под контролем DVC

Рано или поздно нам понадобиться обновлять файлы под DVC-контролем.

Покажем как это делается:

In [14]:
# Дополним наш датасет новыми данными:
!echo '6;0.31;0.47;3.6;0.067;18;42;0.99549;3.39;0.66;11;6' >> 'winequality-red.csv'

dvc заметит, что файлы изменились:

In [15]:
!dvc status

winequality-red.csv.dvc:
	changed outs:
		modified:           winequality-red.csv


Добавим изменения в кэш DVC:

In [16]:
!dvc add winequality-red.csv

Saving 'winequality-red.csv' to '.dvc/cache/9e/55c66554e24691183fa3a2e5ae3c9d'.
Saving information to 'winequality-red.csv.dvc'.

To track the changes with git, run:

	git add winequality-red.csv.dvc


**ВАЖНО!!!**

Обновлять файлы нужно осторожно. Если в DVC изменяли параметр cache.type на значения hardlink и/или symlink (значения по умолчанию - reflink,copy), то нужно сначала убрать файл из под контроля DVC. Подробнее см. https://dvc.org/doc/user-guide/update-tracked-file

## Создание DVC-пайплайнов

В этом разделе мы:
- Обучим нашу модель
- Дадим другим пользователям возможность воспроизвести результаты экспериментов без переобучения всей модели

In [17]:
import sklearn

In [18]:
%%bash
dvc run \
    -f train_model.dvc \
    -d train_model.py -d 'winequality-red.csv' \
    -o 'mymodel-pickle.sav' \
    -M metrics.txt \
    python train_model.py \
        --input_csv_path 'winequality-red.csv' \
        --output_checkpoint_path 'mymodel-pickle.sav' \
        --output_metrics_path 'metrics.txt'

Running command:
	python train_model.py --input_csv_path winequality-red.csv --output_checkpoint_path mymodel-pickle.sav --output_metrics_path metrics.txt
Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')
Training model...
Evaluating metrics...
R-score is 0.3324427990501123
Adding 'mymodel-pickle.sav' to '.gitignore'.
Output 'metrics.txt' doesn't use cache. Skipping saving.
Saving 'mymodel-pickle.sav' to '.dvc/cache/f3/300f01ed0c928dd0e4020b97b335f2'.
Saving information to 'train_model.dvc'.

To track the changes with git, run:

	git add .gitignore train_model.dvc


Рассмотрим созданный пайплайн:

In [19]:
!dvc pipeline show train_model.dvc

winequality-red.csv.dvc
train_model.dvc


Пайплайн для любителей ascii art:

In [20]:
%%bash
dvc pipeline show --ascii train_model.dvc

+-------------------------+  
| winequality-red.csv.dvc |  
+-------------------------+  
              *              
              *              
              *              
    +-----------------+      
    | train_model.dvc |      
    +-----------------+      


Посмотрим, что изменилось:

In [21]:
!git status

На ветке master
Ваша ветка обновлена в соответствии с «origin/master».

Изменения, которые не в индексе для коммита:
  (используйте «git add <файл>…», чтобы добавить файл в индекс)
  (используйте «git checkout -- <файл>…», чтобы отменить изменения
   в рабочем каталоге)

	изменено:      .gitignore
	изменено:      winequality-red.csv.dvc

Неотслеживаемые файлы:
  (используйте «git add <файл>…», чтобы добавить в то, что будет включено в коммит)

	metrics.txt
	train_model.dvc

нет изменений добавленных для коммита
(используйте «git add» и/или «git commit -a»)


Сохраним наш прогресс:

In [22]:
%%bash
git add .
git commit -m "Training pipeline"

# Запушить в DVC:
dvc push -r myremote

# Запушить в git:
git push

[master e65d6d1] Training pipeline
 4 files changed, 24 insertions(+), 2 deletions(-)
 create mode 100644 metrics.txt
 create mode 100644 train_model.dvc
Preparing to upload data to '../dvc_remote'
Preparing to collect status from ../dvc_remote
[##############################] 100% 

[##############################] 100% 
[##############################] 100% Analysing status
[##############################] 100% mymodel-pickle.sav
[##############################] 100% mymodel-pickle.sav
[##############################] 100% winequality-red.csv
[##############################] 100% winequality-red.csv


To /home/nik/tmp_dvc/git_remote
   a803109..e65d6d1  master -> master


# Боб

Давайте воспроизведем построенный пайплайн у Боба

Перейти в каталог Боба:

In [23]:
%cd $BOB_DIR

/home/nik/tmp_dvc/Bob


Клонируем репозиторий

In [24]:
!git clone $GIT_REMOTE $BOB_DIR

Клонирование в «/home/nik/tmp_dvc/Bob»…
готово.


Получаем данные из dvc:

In [25]:
!dvc pull -r myremote

Preparing to download data from '../dvc_remote'
Preparing to collect status from ../dvc_remote
[##############################] 100% 

[##############################] 100% 
[##############################] 100% Analysing status
[##############################] 100% winequality-red.csv
[##############################] 100% winequality-red.csv
[##############################] 100% mymodel-pickle.sav
[##############################] 100% mymodel-pickle.sav
[##############################] 100% Checkout finished!


При выполнении `dvc pull` команда `dvc` проходить по файлам `*.dvc` и по прописанным внутри них хэшам понимает какие файлы нужно скачать.

Убедимся, что у нас актуальная версия данных:

In [26]:
!dvc status

Pipeline is up to date. Nothing to reproduce.


Воспроизведем результаты первого пользователя:

In [27]:
!dvc repro train_model.dvc

Stage 'winequality-red.csv.dvc' didn't change.
Stage 'train_model.dvc' didn't change.
Pipeline is up to date. Nothing to reproduce.


DVC обнаружил, что все текущие данные актуальны и переобучать модель не надо. 
Если бы требовалось переобучить модель, то dvc бы запустил скрипт для обучения.